<a href="https://colab.research.google.com/github/L120Git/Deep_Learning_Airbnb/blob/main/Deep_Learning_LauraCast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q keras

In [ ]:
import numpy as np
import pandas as pd
#para representar la matriz de correlación
import seaborn as sns
#para descargar las imagenes
import imageio as io
import cv2
#para representar gráficas
import matplotlib.pyplot as plt

#para preprocesar
from sklearn import preprocessing
#para dividir los datos
from sklearn.model_selection import train_test_split
#para entrar redes
import tensorflow as tf
from tensorflow import keras
#convertir un vector en una matriz binaria
from keras.utils import to_categorical #no lo uso
#para evaluar
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Nos descargamos el dataset de OpenDataSoft
!wget -O "airbnb-listings.csv" "https://public.opendatasoft.com/explore/dataset/airbnb-listings/download/?format=csv&disjunctive.host_verifications=true&disjunctive.amenities=true&disjunctive.features=true&refine.country=Spain&q=Madrid&timezone=Europe/London&use_labels_for_header=true&csv_separator=%3B"

--2023-02-28 11:51:26--  https://public.opendatasoft.com/explore/dataset/airbnb-listings/download/?format=csv&disjunctive.host_verifications=true&disjunctive.amenities=true&disjunctive.features=true&refine.country=Spain&q=Madrid&timezone=Europe/London&use_labels_for_header=true&csv_separator=%3B
Resolving public.opendatasoft.com (public.opendatasoft.com)... 34.248.20.69, 34.249.199.226
Connecting to public.opendatasoft.com (public.opendatasoft.com)|34.248.20.69|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/csv]
Saving to: ‘airbnb-listings.csv’

airbnb-listings.csv     [   <=>              ]  54.31M   616KB/s    in 1m 44s  

2023-02-28 11:53:21 (533 KB/s) - ‘airbnb-listings.csv’ saved [56950929]



In [ ]:
# Leemos el dataset con pandas y mostramos los 5 primeros registros
data = pd.read_csv('airbnb-listings.csv', sep = ';')
data.head()

,ID,Listing Url,Scrape ID,Last Scraped,Name,Summary,Space,Description,Experiences Offered,Neighborhood Overview,...,Review Scores Communication,Review Scores Location,Review Scores Value,License,Jurisdiction Names,Cancellation Policy,Calculated host listings count,Reviews per Month,Geolocation,Features
0,15141125,https://www.airbnb.com/rooms/15141125,20170407214119,2017-04-08,Panoramic charming studio!,"Charming bright and cosy studio in ""El Rastro""...",It's a bright and cosy studio located in the h...,"Charming bright and cosy studio in ""El Rastro""...",none,The studio is located in the remarkable neighb...,...,10.0,10.0,10.0,NaN,NaN,moderate,2,5.11,"40.40745268477803,-3.7079954754432634","Host Is Superhost,Host Has Profile Pic,Host Id..."
1,9470166,https://www.airbnb.com/rooms/9470166,20170407214119,2017-04-08,Comfortable penthouse in the center,"In the center, recently remodeled, comfortable...",Quiet and comfortable space where you can rela...,"In the center, recently remodeled, comfortable...",none,It belongs to the district center of Madrid wh...,...,9.0,9.0,8.0,NaN,NaN,strict,13,5.88,"40.40831556101927,-3.705006385704201","Host Has Profile Pic,Host Identity Verified,Re..."
2,17444981,https://www.airbnb.com/rooms/17444981,20170407214119,2017-04-08,Cosy Apartment SOL/LA LATINA,Nice apartment near Sol and La Latina. Perfect...,Is a really cosy apartment with the best locat...,Nice apartment near Sol and La Latina. Perfect...,none,"La Latina is well known by El Rastro, the most...",...,10.0,10.0,10.0,NaN,NaN,moderate,1,6.18,"40.413722014584295,-3.7068231186536718","Host Has Profile Pic,Host Identity Verified,Re..."
3,3284565,https://www.airbnb.com/rooms/3284565,20170407214119,2017-04-08,Alquiler Estudio Nuevo Madrid,"Alquilo estudio en el centro de Madrid , total...",NaN,"Alquilo estudio en el centro de Madrid , total...",none,NaN,...,10.0,9.0,9.0,NaN,NaN,strict,3,0.19,"40.411263372718054,-3.6973054373712886","Host Has Profile Pic,Host Identity Verified,Is..."
4,499911,https://www.airbnb.com/rooms/499911,20170407214119,2017-04-08,2 bedroom apartment downtown Madrid,NaN,Bright and airy apartment in Central Madrid re...,Bright and airy apartment in Central Madrid re...,none,NaN,...,10.0,10.0,10.0,NaN,NaN,strict,1,1.39,"40.41255625665078,-3.701155364799422","Host Is Superhost,Host Has Profile Pic,Host Id..."


In [ ]:
# vamos a ver de cuantos registros disponemos y de cuantas variables
data.shape

(14001, 89)

In [ ]:
# ¿De qué variables disponemos?
data.columns

Index(['ID', 'Listing Url', 'Scrape ID', 'Last Scraped', 'Name', 'Summary',
       'Space', 'Description', 'Experiences Offered', 'Neighborhood Overview',
       'Notes', 'Transit', 'Access', 'Interaction', 'House Rules',
       'Thumbnail Url', 'Medium Url', 'Picture Url', 'XL Picture Url',
       'Host ID', 'Host URL', 'Host Name', 'Host Since', 'Host Location',
       'Host About', 'Host Response Time', 'Host Response Rate',
       'Host Acceptance Rate', 'Host Thumbnail Url', 'Host Picture Url',
       'Host Neighbourhood', 'Host Listings Count',
       'Host Total Listings Count', 'Host Verifications', 'Street',
       'Neighbourhood', 'Neighbourhood Cleansed',
       'Neighbourhood Group Cleansed', 'City', 'State', 'Zipcode', 'Market',
       'Smart Location', 'Country Code', 'Country', 'Latitude', 'Longitude',
       'Property Type', 'Room Type', 'Accommodates', 'Bathrooms', 'Bedrooms',
       'Beds', 'Bed Type', 'Amenities', 'Square Feet', 'Price', 'Weekly Price',
       'Month

Voy a filtrar por Madrid el dataset porque he observado errores en 'City'


In [ ]:
#observo si hay filas que no pertenecen a Madrid
print(f'Hay', (data['City'] != 'Madrid').sum(), 'filas que no son de Madrid')

Hay 794 filas que no son de Madrid


In [ ]:
#elimino esas filas
data_mad = data.loc[data['City']=='Madrid']
#compruebo que las he eliminado
(data_mad['City'] != 'Madrid').sum()

0

In [ ]:
# vamos a ver las medidas del data_mad
data_mad.shape

(13207, 89)

In [ ]:
#reseteo los indices para que no haya problemas después al descargar las imagenes
data_mad = data_mad.reset_index()

Cargamos las imágenes del entrenamiento en un array para tenerlas disponibles luego

In [ ]:
# montamos GDrive para almacenar las imagenes que descarguemos
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
n_images = 1000
images = np.zeros((n_images, 224, 224, 3), dtype=np.uint8)
urls = data_mad['Thumbnail Url']

i_aux = 0
good_urls = []
for i_img, url in enumerate(urls):
    if len(good_urls) >= n_images:
        # ya tenemos n_images imágenes
        break
    try:
        img = io.imread(url)
        images_aux = cv2.resize(img, (224, 224))
        if images_aux.shape == (224, 224, 3):
          images[i_aux]=images_aux
          good_urls.append(i_img)
          i_aux += 1
          print(f'Imagen {i_img} descargada')
          print(len(good_urls))
        else:
          pass
    except IOError as err:
        pass

Imagen 0 descargada
1
Imagen 1 descargada
2
Imagen 2 descargada
3
Imagen 4 descargada
4
Imagen 7 descargada
5
Imagen 11 descargada
6
Imagen 13 descargada
7
Imagen 15 descargada
8
Imagen 16 descargada
9
Imagen 17 descargada
10
Imagen 21 descargada
11
Imagen 23 descargada
12
Imagen 24 descargada
13
Imagen 25 descargada
14
Imagen 26 descargada
15
Imagen 27 descargada
16
Imagen 29 descargada
17
Imagen 30 descargada
18
Imagen 31 descargada
19
Imagen 32 descargada
20
Imagen 34 descargada
21
Imagen 35 descargada
22
Imagen 36 descargada
23
Imagen 37 descargada
24
Imagen 38 descargada
25
Imagen 39 descargada
26
Imagen 40 descargada
27
Imagen 41 descargada
28
Imagen 43 descargada
29
Imagen 45 descargada
30
Imagen 46 descargada
31
Imagen 47 descargada
32
Imagen 48 descargada
33
Imagen 52 descargada
34
Imagen 54 descargada
35
Imagen 55 descargada
36
Imagen 56 descargada
37
Imagen 57 descargada
38
Imagen 58 descargada
39
Imagen 59 descargada
40
Imagen 60 descargada
41
Imagen 61 descargada
42
Imagen

KeyboardInterrupt: ignored

In [ ]:
# Visualizamos las imágenes cargadas
print(images.shape)

In [ ]:
# Mantenemos los datos numéricos solo para aquellos pisos que tienen imágenes 
# y las hemos obtenido
final_data = data_mad.iloc[good_urls, :]

In [ ]:
print(final_data.shape)

In [ ]:
# guardamos las imágenes (y yo os recomiendo que os lo guardéis en GDrive para evitar tener que repetir esto)
np.save('images.npy', images)
np.save('final_data.npy', final_data)
np.save('good_url.npy', good_urls)

In [ ]:
# almacenamos las imagenes en nuestro drive
!cp images.npy /content/drive/MyDrive/DL/images.npy
!cp final_data.npy /content/drive/MyDrive/DL/final_data.npy
!cp good_url.npy /content/drive/MyDrive/DL/good_url.npy


In [ ]:
!ls -lah images.npy final_data.npy good_url.npy

In [ ]:
#Elimino del dataset la variable de las urls porque ya no la necesito

final_data = final_data.drop(['Thumbnail Url'], axis=1)
final_data.head(3)

### División de los datos

In [ ]:
#Divido entre train y test
train, test = train_test_split(final_data, test_size=0.2, shuffle=True, random_state=42)
print(f'Dimensiones del dataset de training: {train.shape}')
print(f'Dimensiones del dataset de test: {test.shape}')

#guardo los CSV en local
train.to_csv('./train.csv', sep=';', decimal='.', index=False)
test.to_csv('./test.csv', sep=';', decimal='.', index=False)

In [ ]:
#compruebo que tengo los indices originales
train.head()

In [ ]:
test.head()

### Cargamos los datos de train 

In [ ]:
#cargo los datos de train para trabajar con ellos
#df_train = pd.read_csv('./train.csv', sep=';', decimal='.') #OJO! si cargo el csv me cambia los indices porque lee de nuevo y hace un df del csv
df_train = train
df_train.shape

### Análisis exploratorio de train

In [ ]:
#eliminar variables irrelevantes para el modelo, las 4 últimas son fruto de una regularización anterior por lasso
df_train = df_train.drop(['Neighbourhood','City','Zipcode','ID','Listing Url','Scrape ID','Last Scraped',
                                  'Name', 'Summary','Space','Description','Experiences Offered','Neighborhood Overview',
                                  'Notes','Amenities','Square Feet','Transit','Access' ,'Interaction', 'House Rules' ,
                                  'Medium Url','Picture Url','XL Picture Url','Host ID', 'Host URL', 
                                  'Host Name', 'Host Since', 'Host Location', 'Host About','Host Response Time',
                                  'Host Response Rate','Host Acceptance Rate','Host Thumbnail Url','Host Picture Url', 
                                  'Host Neighbourhood','Host Listings Count','Host Total Listings Count','Host Verifications',
                                  'Street','Neighbourhood Group Cleansed','State','Market','Smart Location','Country Code',
                                  'Country','Latitude','Longitude', 'Security Deposit','Cleaning Fee',
                                  'Minimum Nights','Maximum Nights','Calendar Updated','Has Availability','Availability 30',
                                  'Availability 60','Availability 90','Availability 365','Calendar last Scraped', 
                                  'First Review','Last Review','Review Scores Accuracy','Review Scores Cleanliness',
                                  'Review Scores Checkin','Weekly Price', 'Monthly Price','Review Scores Communication',
                                  'Review Scores Location','Review Scores Value','License','Jurisdiction Names',
                                  'Calculated host listings count','Reviews per Month','Geolocation','Features','Guests Included', 
                                  'Cancellation Policy','Bed Type','Property Type' ], axis=1)
df_train.columns

In [ ]:
#renombro las columnas
df_train.columns=['index','Neighbourhood_Cleansed', 'Room_Type','Accommodates',
                  'Bathrooms', 'Bedrooms', 'Beds','Price','Extra_People',
                      'Number_of_Reviews', 'Review_Scores_Rating']

In [ ]:
#búsqueda de valores nulos
df_train.isnull().any()

In [ ]:
#contabilizo los valores nulos
print(f'Bathrooms tiene: ', df_train['Bathrooms'].isnull().sum())
print(f'Bedrooms tiene: ', df_train['Bedrooms'].isnull().sum())
print(f'Beds tiene: ', df_train['Beds'].isnull().sum())
print(f'Review_Scores_Rating tiene: ', df_train['Review_Scores_Rating'].isnull().sum())

In [ ]:
#Imputación de valores nulos por la moda
df_train["Bathrooms"].fillna(df_train["Bathrooms"].mode()[0], inplace=True)
df_train["Bedrooms"].fillna(df_train["Bedrooms"].mode()[0], inplace=True)
df_train["Beds"].fillna(df_train["Beds"].mode()[0], inplace=True)
df_train["Review_Scores_Rating"].fillna(df_train["Review_Scores_Rating"].mode()[0], inplace=True) 
#compruebo
print(f'Bathrooms tiene: ', df_train['Bathrooms'].isnull().sum())
print(f'Bedrooms tiene: ', df_train['Bedrooms'].isnull().sum())
print(f'Beds tiene: ', df_train['Beds'].isnull().sum())
print(f'Review_Scores_Rating tiene: ', df_train['Review_Scores_Rating'].isnull().sum())

In [ ]:
#codificación de variables categóricas
df_train.dtypes

In [ ]:
#utilizo MeanEncoder para imputar con la media de la variable objetivo, menos las imagenes
categorical = ['Neighbourhood_Cleansed', 'Room_Type']

mean_map = {}
for c in categorical:
    mean = df_train.groupby(c)['Price'].mean()
    df_train[c] = df_train[c].map(mean)    
    mean_map[c] = mean
df_train.dtypes

In [ ]:
df_train

In [ ]:
#Observación y limpieza de outlaiers

#scatter Bedrooms x Price
df_train.plot(kind = 'scatter',x='Bedrooms',y = 'Price')
plt.xlabel('Bedrooms')
plt.ylabel('Price')
plt.show()

#boxplot Bathrooms x Price
df_train.plot(kind = 'scatter',x='Bathrooms',y = 'Price')
plt.xlabel('Bathrooms')
plt.ylabel('price')
plt.show()

#boxplot Beds x Price
df_train.boxplot(by='Beds',column = 'Price')
plt.show()

#boxplot Accommodates x Price
df_train.plot(kind = 'scatter',x='Accommodates',y = 'Price')
plt.xlabel('Accommodates')
plt.ylabel('price')
plt.show()


In [ ]:
#aplico filtros para reducir outliers
df_train_filter = df_train[(df_train['Bedrooms'] > 0) & (df_train['Bedrooms'] < 7)]
df_train_filter2 = df_train_filter[(df_train_filter['Bathrooms'] > 0) & (df_train_filter['Bathrooms'] < 4.5)]

df_train_filter4 = df_train_filter2[(df_train_filter2['Accommodates'] > 1) & (df_train_filter2['Accommodates'] < 11)]

In [ ]:
#análisis de correlación 
df_train_filter4.corr().T

In [ ]:
df_train_filter4.index

In [ ]:
# Compute the correlation matrix
corr = np.abs(df_train_filter4.drop(['Price'], axis=1).corr())

# Generate a mask for the upper triangle
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(12, 10))

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask,vmin = 0.0, vmax=1.0, center=0.5,
            linewidths=.1, cmap="YlGnBu", cbar_kws={"shrink": .8})

plt.show()

In [ ]:
#elimino Beds por colinealidad con accommodates
df_training= df_train_filter4.drop(['Beds'],axis=1)

In [ ]:
#veo las variables que me quedan
df_training.columns

In [ ]:
#las nuevas medidas del df
df_training.shape

In [ ]:
df_training.fillna(0, inplace=True) #reemplazo los valores nulos a 0 por si los hubiera.

In [ ]:
df_training.isnull().values.any() #me aseguro de que no hay valores nulos

### División de los datos de entrenamiento, la etiqueta y 'Price' por un lado, el resto por otro.

In [ ]:
#Divido en X e y los datos del df
X_train =df_training.drop(['Price'], axis=1) # me defino un df sin la variable objetivo

# Obtener las etiquetas de regresion y_reg que la voy a llamar y_train
Y_train = df_training['Price'] #esta es mi variable objetivo

print('Filas, columnas', X_train.shape)
print('Filas, columnas', Y_train.shape)

## Cargo las imagenes de train para después

In [ ]:
#Busco las imagenes de los train para el modelo CNN
images = np.load('./images.npy')
good_url = np.load('./good_url.npy')
indices_train= X_train.index #obtengo los indices de mi conjunto de entrenamiento

#me paso good_url a array #(podía haber lo hecho con good_urls que ya era una lista)
index_ = [] 
for elemento in good_url:
    index_.append(elemento)

#voy a buscar los indices de las imagenes que tengo en mi X_train
images_train_index= []

for elemento in indices_train:
   images_train_index.append(elemento)

#comparo los indices de train con good_url y asocio las imagenes 
imagenes_train=[]

for elemento in indices_train:
  for posicion, indice in enumerate(index_):
      if elemento == indice:
          imagenes_train.append(images[posicion])



In [ ]:
indices_not_in_index_= []
def conjuntos(X_train,index):
  for a in X_train.index:
    if a not in index_:
      indices_not_in_index_.append(a)
      return False
  a +=1
  return True
conjuntos(X_train,index_)

In [ ]:
len(imagenes_train)

In [ ]:
imagenes_train

### Preparo y normalizo los datos de train

In [ ]:
from sklearn.preprocessing import MinMaxScaler
#Normalizo las etiquetas con el máximo
Y_train_norm = Y_train /Y_train.max() 

#normalizo el resto de los datos con StandarScaler
scaler = MinMaxScaler(feature_range=(0, 1))
X_train_norm = scaler.fit_transform(X_train)


In [ ]:
Y_train_norm

In [ ]:
X_train_norm

### Cargamos los datos de test

In [ ]:
#df_test = pd.read_csv('./test.csv', sep=';', decimal='.') #OJO! si cargo el csv me cambia los indices porque lee de nuevo y hace un df del csv
df_test = test
df_test.head()

### Aplicamos los mismos filtros que he aplicado en train

In [ ]:
#elimino las filas que no son de Madrid
df_test = df_test.loc[df_test['City']=='Madrid']
#compruebo que las he eliminado
(df_test['City'] != 'Madrid').sum()

In [ ]:
#eliminar variables irrelevantes para el modelo
df_test = df_test.drop(['Neighbourhood','City','Zipcode','ID','Listing Url','Scrape ID','Last Scraped',
                                  'Name', 'Summary','Space','Description','Experiences Offered','Neighborhood Overview',
                                  'Notes','Amenities','Square Feet','Transit','Access' ,'Interaction', 'House Rules' ,
                                  'Medium Url','Picture Url','XL Picture Url','Host ID', 'Host URL', 
                                  'Host Name', 'Host Since', 'Host Location', 'Host About','Host Response Time',
                                  'Host Response Rate','Host Acceptance Rate','Host Thumbnail Url','Host Picture Url', 
                                  'Host Neighbourhood','Host Listings Count','Host Total Listings Count','Host Verifications',
                                  'Street','Neighbourhood Group Cleansed','State','Market','Smart Location','Country Code',
                                  'Country','Latitude','Longitude', 'Security Deposit','Cleaning Fee',
                                  'Minimum Nights','Maximum Nights','Calendar Updated','Has Availability','Availability 30',
                                  'Availability 60','Availability 90','Availability 365','Calendar last Scraped', 
                                  'First Review','Last Review','Review Scores Accuracy','Review Scores Cleanliness',
                                  'Review Scores Checkin','Weekly Price', 'Monthly Price','Review Scores Communication',
                                  'Review Scores Location','Review Scores Value','License','Jurisdiction Names',
                                  'Calculated host listings count','Reviews per Month','Geolocation','Features','Guests Included', 
                                  'Cancellation Policy','Bed Type','Property Type' ], axis=1)
df_train.columns

In [ ]:
#renombro las columnas
df_test.columns=['index','Neighbourhood_Cleansed', 'Room_Type','Accommodates',
                  'Bathrooms', 'Bedrooms', 'Beds','Price','Extra_People',
                      'Number_of_Reviews', 'Review_Scores_Rating']

In [ ]:
#Imputación de valores nulos por la moda
df_test["Bedrooms"].fillna(df_test["Bedrooms"].mode()[0], inplace=True)
df_test["Review_Scores_Rating"].fillna(df_test["Review_Scores_Rating"].mode()[0], inplace=True) 
#compruebo
print(f'Bedrooms tiene: ', df_test['Bedrooms'].isnull().sum())
print(f'Review_Scores_Rating tiene: ', df_test['Review_Scores_Rating'].isnull().sum())

In [ ]:
#compruebo si hay más valores nulos 
df_test.isnull().any()

In [ ]:
#utilizo MeanEncoder para imputar con la media de la variable objetivo
categorical = ['Neighbourhood_Cleansed', 'Room_Type']

mean_map = {}
for c in categorical:
    mean = df_test.groupby(c)['Price'].mean()
    df_test[c] = df_test[c].map(mean)    
    mean_map[c] = mean
df_test.dtypes

In [ ]:
#aplico filtros para reducir outliers
df_test_filter = df_test[(df_test['Bedrooms'] > 0) & (df_test['Bedrooms'] < 7)]
df_test_filter2 = df_test_filter[(df_test_filter['Bathrooms'] > 0) & (df_test_filter['Bathrooms'] < 4.5)]

df_test_filter4 = df_test_filter2[(df_test_filter2['Accommodates'] > 1) & (df_test_filter2['Accommodates'] < 11)]

In [ ]:
#elimino Beds por colinealidad con accommodates
df_testing= df_test_filter4.drop(['Beds'],axis=1)

In [ ]:
#veo las variables que me quedan
df_testing.columns

In [ ]:
#las nuevas medidas del df
df_testing.shape

In [ ]:
df_testing.fillna(0, inplace=True) #reemplazo los valores nulos a 0 por si los hubiera.

In [ ]:
df_testing.isnull().values.any()

### Split en X e y  de los datos de test

In [ ]:
#excluyo las etiquetas del split
X_test =df_testing.drop(['Price'], axis=1) # me defino un df sin la variable objetivo

# Obtener las etiquetas de regresion y_reg que la voy a llamar y_test
Y_test = df_testing['Price'] #esta es mi variable objetivo

print('Filas, columnas', X_test.shape)
print('Filas, columnas', Y_test.shape)

## me saco las imagenes para luego

In [ ]:
#Busco las imagenes de test para el modelo CNN

#images = np.load('./images.npy') ya está cargado
#good_url = np.load('./good_url.npy') ya está cargado
indices_test= X_test.index

#me paso good_url a lista (podía haber lo hecho con good_urls que ya era una lista)
index_tst_ = [] 
for elemento in good_url:
    index_tst_.append(elemento)
  

#voy a buscar los indices de las imagenes que tengo en mi X_train
images_test_index= []

for elemento in indices_test:
    images_test_index.append(elemento)



#comparo los indices de train con good_url y asocio las imagenes 
imagenes_test=[]

for elemento in indices_test:
  for posicion, indice in enumerate(index_tst_):
      if elemento == indice:
          imagenes_test.append(images[posicion])




In [ ]:
len(imagenes_test)

### normalizamos datos de test

In [ ]:
# Normalizamos los datos (sin la variable objetivo), entre 0 y 1
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0, 1))
X_test_norm = scaler.fit_transform(X_test) #estandarizo con el fit de train

y_test_norm = Y_test /Y_train.max() #estandarizo el precio con el max del precio en train

#simplifico el nombre con los datos normalizados, hago este paso para tener claro que he normalizado antes
x_train = X_train_norm
y_train = Y_train_norm
x_test = X_test_norm
y_test = y_test_norm

##### Compruebo que todos mis datos están normalizados entre 0 y 1

In [ ]:
x_train

In [ ]:
y_train

In [ ]:
x_test

In [ ]:
y_test

### Compruebo medidas del dataset 


In [ ]:
# Comprobar el tamaño del dataset
print("El conjunto de entrenamiento tiene dimensiones: ", x_train.shape)
#print("El conjunto de validación tiene dimensiones: ",x_val.shape)
print("El conjunto de test tiene dimensiones: ",x_test.shape)

## Modelo con datos 1D

### Importamos librerias

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from sklearn.metrics import classification_report
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.optimizers.legacy import SGD
from tensorflow.keras.optimizers.legacy import Adam
import matplotlib.pyplot as plt
from pandas.core.common import flatten
from tensorflow.keras import optimizers, Model

In [ ]:
# Hiper-parámetros de nuestra red, pruebo distintos parámetros hasta mejorar mi modelo

#lr = 0.1
#lr_decay = 1e-4
#n_epochs = 200  
#batch_size = 128

#cambio los parámetros para reducir el consumo de memoria y que tarde menos en entrenar
lr = 0.1
lr_decay = 1e-4
n_epochs = 100
batch_size = 64

#vuelvo a mejorar los parámetros para que aun me tarde menos al entrenar

#lr = 0.05
#lr_decay = 1e-4
#n_epochs = 50
#batch_size = 32


In [ ]:
x_train.shape #este es mi input shape

In [ ]:
np.random.seed(42)
# Implementamos la red empleando Keras
lineal_model = Sequential() # Instancia de modelo API secuencial #(X)
#model.add(Flatten()) # Estiramos los datos en forma de vector como entrada 
#model.add(Dense(128, activation="relu", kernel_initializer='random_uniform', bias_initializer='zeros'))
lineal_model.add(Dense(64, input_shape=(9,),activation="relu")) 
lineal_model.add(Dense(32, activation="relu")) # Añadimos una capa con 32 neuronas y añadimos una función de activacion
lineal_model.add(Dense(1, activation="sigmoid"))  # Construimos nuestro Perceptron
  
# Compilamos y entrenamos el modelo SGD
print("[INFO]: Entrenando red neuronal...")
lineal_model.compile(loss="mean_squared_error", optimizer=SGD(lr, decay=lr_decay/n_epochs))  #(X) optimizer='adam'

# Entrenando la solución
H_lm = lineal_model.fit(x_train, y_train, validation_split=0.2 , epochs=n_epochs, batch_size=batch_size)  #(X) aquí divido los datos de validacion

# Genero la predicción con los datos estandarizados de test
pred_lm = lineal_model.predict(x_test, batch_size=batch_size)


lineal_model.summary()
# Muestro gráfica de losses
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, n_epochs), H_lm.history["loss"], label="train_loss")
plt.plot(np.arange(0, n_epochs), H_lm.history["val_loss"], label="val_loss")
plt.title("Training Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()

In [ ]:
#calculo el mse y el mae del modelo
max_Y = Y_train.max()
pred_lm_denorm = pred_lm * max_Y
y_test_denorm = y_test * max_Y

rmse = mean_squared_error(y_true  = y_test_denorm, y_pred  = pred_lm_denorm,squared = False)
print(f"El error cuadrático medio (rmse) de test es: {rmse}")

mae = mean_absolute_error(y_test_denorm, pred_lm_denorm)
print(f"El error aboluto(mae) de test es: {mae}")

In [ ]:
#calculamos las perdidas del modelo
test_loss = lineal_model.evaluate(x_test, y_test, verbose=0)
print('test loss: ', test_loss) 

##Modelo CNN

#### Preparación de los datos

In [ ]:
#Preparo los datos 
X_train_img = imagenes_train
X_test_img = imagenes_test

In [ ]:
#Comprobamos que tenemos todas las imagenes de X_Train y X_test cargadas con sus indices correspondientes
print('El número total de imagenes de entrenamiento es: ',len(imagenes_train)) # no son las imagenes correctas
print('El número total de imagenes de testing es: ',len(imagenes_test)) 
print("\n")
print("Las etiquetas de entrenamiento ", y_train.shape)
print("Las etiquetas de testing: ",y_test.shape)

In [ ]:
#mis imagenes son imagenes_train & imagenes_test

In [ ]:
!pip install imageio

In [ ]:
# import necesarios

from keras.models import Sequential
from keras.applications import VGG16, ResNet50
from keras.applications.inception_v3 import preprocess_input
from keras import optimizers, Model
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from keras.optimizers import Adam
from keras.layers import Dropout, Flatten, Dense, Activation
from keras.applications import imagenet_utils
import imageio as io

#inicializo con una semilla
np.random.seed(42)

input_shape = (224, 224, 3)


#X_train = imagenes_train
#X_test = imagenes_test
#y_train = y_train
#y_test = y_test

In [ ]:
#Realizamos el resize de las imagenes
input_shape = (224, 224,3)

 #resize train set
X_train_resized = []
for img in x_train:
  X_train_resized.append( cv2.resize(img, (224, 224))/255)
  
X_train_resized = np.array(X_train_resized)
print('X_train_resized: ',X_train_resized.shape)


 #resize test set
X_test_resized = []
for img in x_test:
  X_test_resized.append( cv2.resize(img, (224, 224))/255)
  
X_test_resized = np.array(X_test_resized)
print('X_test_resized: ',X_test_resized.shape)

In [ ]:
# normalizamos
new_width = 224
new_height = 224
dim = (new_width, new_height)

# Inicializa una lista para almacenar las imágenes redimensionadas y normalizadas
X_train_resized = []

# Recorre cada imagen en x_train
for img in imagenes_train:
    # Redimensiona la imagen
    resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
    
    # Convierte la imagen en un array numpy y normaliza los valores de los pixeles
    image_array = np.array(resized, dtype='float32') / 255.0

    # Agrega la imagen redimensionada y normalizada a la lista
    X_train_resized.append(image_array)
    
#Realizamos lo mismo para los datos test

X_test_resized = []

for img in imagenes_test:
    
    resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)

    # Convierte la imagen en un array numpy y normaliza los valores de los pixeles
    image_array = np.array(resized, dtype='float32') / 255.0 

    # Agrega la imagen redimensionada y normalizada a la lista
    X_test_resized.append(image_array)
   

In [ ]:
 print((X_train_resized))

In [ ]:
print((X_test_resized))

In [ ]:
plt.imshow(X_test_resized[12])

#### Comienzo con un Trasfer Learning de VGG16 (mis muestras son poco más de 500) con todas las capas congeladas

In [ ]:
np.random.seed(42)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
base_model.summary()

In [ ]:
#Congelamos todas las capas de extración de características y modificamos el clasificador
for layer in base_model.layers: 
  layer.trainable = False
  print('Capa ' + layer.name + ' congelada.')

# cogemos la última capa del model y le añadimos nuestro clasificador (top_model)
last = base_model.layers[-1].output
x = Flatten()(last) #globalMaxPooling #pasar el tensor de caracteristicas a vector
x = Dense(1000, activation='relu', name='fc1')(x)
x = Dropout(0.3)(x) # regularización para reducir el sobreajuste
x = Dense(1, activation='linear', name='predictions')(x) #linear multiplica por x1.0, es la indentity function
model_cnn = Model(base_model.input, x)

# compilamos el modelo
model_cnn.compile(optimizer=Adam(lr=0.001), loss='mean_squared_error', metrics=['mse'])

# vemos la estructura del modelo
model_cnn.summary()

In [ ]:
X_train_resized = np.array(X_train_resized)
X_test_resized = np.array(X_test_resized)

In [ ]:
# comenzamos con el entrenamiento
epochs = 20
batch_size = 128

# como entrenariamos normalmente
H_model_cnn = model_cnn.fit(X_train_resized, Y_train,
          batch_size=batch_size,
          validation_split=0.2,
          epochs=epochs)

# Evaluamos el modelo
print("[INFO]: Evaluando red neuronal...")
pred_cnn = model_cnn.predict(X_test_resized, batch_size=batch_size)  #(X)

model_cnn.summary()

#grafico
sns.set(style="darkgrid")
plt.figure(figsize=(6, 4))
plt.plot(H_model_cnn.history['loss'], label='Train')
plt.plot(H_model_cnn.history['val_loss'], label='Validation')
plt.title('Model_cnn Loss', fontsize=18)
plt.ylabel('Loss', fontsize=12)
plt.xlabel('Epoch', fontsize=12)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.legend(loc='middle right', fontsize=10)
sns.despine()
plt.show()

In [ ]:
#calculo el mse y el mae del modelo
max_Y = Y_train.max()
pred_cnn_denorm = pred_cnn 
y_test_denorm = y_test * max_Y

rmse = mean_squared_error(y_true  = y_test_denorm, y_pred  = pred_cnn_denorm,squared = False)
print(f"El error cuadrático medio (rmse) de test es: {rmse}")

mae = mean_absolute_error(y_test_denorm, pred_cnn_denorm)
print(f"El error aboluto(mae) de test es: {mae}")


In [ ]:
#calculamos las perdidas del modelo
scores = model_cnn.evaluate(X_test_resized, Y_test, verbose=1)
print('Test loss:', scores[0])


#### Empiezo a descongelar capas

#### voy a aplicar Fine Tuning 

In [ ]:
#inicializo con una semilla
np.random.seed(42)
input_shape = (224, 224, 3)

bm_FT = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
bm_FT.summary()

In [ ]:
np.random.seed(42)
# Hago Fine Tuning, Descongelo la ultima capa convolucional
from keras.regularizers import l2

for layer in bm_FT.layers:
  if layer.name == 'block5_conv1': #descongelo el bloque 5
    break
  layer.trainable = False
  print('Capa ' + layer.name + ' congelada.')

# cogemos la última capa del model y le añadimos nuestro clasificador (top_model)
last = bm_FT.layers[-1].output
x = Flatten()(last)
x = Dense(1000, activation='relu', name='fc1')(x) 
x = Dropout(0.3)(x)
x = Dense(1, activation='linear', name='predictions')(x)#linear multiplica por x1.0, es la indentity function
model_FT = Model(bm_FT.input, x)

# compilamos el modelo
model_FT.compile(optimizer=Adam(lr=0.001), loss='mse', metrics=['mse'])

# vemos la nueva estructura del modelo
model_FT.summary()

In [ ]:
#entreno de nuevo

# como entrenariamos normalmente
H_model_FT = model_FT.fit(X_train_resized, Y_train,
          batch_size=batch_size,
          validation_split=0.2,
          epochs=20)

# Evaluamos el modelo
print("[INFO]: Evaluando red neuronal...")
pred_FT = model_FT.predict(X_test_resized, batch_size=128)  #(X)

model_FT.summary()

#grafico
sns.set(style="darkgrid")
plt.figure(figsize=(6, 4))
plt.plot(H_model_FT.history['loss'], label='Train')
plt.plot(H_model_FT.history['val_loss'], label='Validation')
plt.title('Model_FT Loss', fontsize=18)
plt.ylabel('Loss', fontsize=12)
plt.xlabel('Epoch', fontsize=12)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.legend(loc='middle right', fontsize=10)
sns.despine()
plt.show()

In [ ]:
#calculo el mse y el mae del modelo
max_Y = Y_train.max()
pred_FT_denorm = pred_FT
y_test_denorm = y_test * max_Y

rmse = mean_squared_error(y_true  = y_test_denorm, y_pred  = pred_FT_denorm,squared = False)
print(f"El error cuadrático medio (rmse) de test es: {rmse}")

mae = mean_absolute_error(y_test_denorm, pred_FT_denorm)
print(f"El error aboluto(mae) de test es: {mae}")


In [ ]:
# comprobamos el loss en el test set
scores = model_FT.evaluate(X_test_resized, Y_test, verbose=1)
print('Test loss:', scores[0])



##### voy a modificar los parámetros a ver si mejora

##### Pruebo a descongelar el penúltimo bloque a ver si mejora

In [ ]:
#inicializo con una semilla
np.random.seed(42)
input_shape = (224, 224, 3)

bm_FT2 = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
bm_FT2.summary()

In [ ]:
# Descongelo otra capa convolucional
#from keras.regularizers import l2

for layer in bm_FT2.layers:
  if layer.name == 'block4_conv1': 
    break
  layer.trainable = False
  print('Capa ' + layer.name + ' congelada.')

# cogemos la penúltima capa del model y le añadimos nuestro clasificador (top_model)
last = bm_FT2.layers[-1].output
x = Flatten()(last)
x = Dense(1000, activation='relu', name='fc1')(x)
x = Dropout(0.3)(x)
x = Dense(1, activation='linear', name='predictions')(x)#linear multiplica por x1.0, es la indentity function
model_FT2 = Model(bm_FT2.input, x)

# compilamos el modelo
model_FT2.compile(optimizer=Adam(lr=0.001), loss='mean_squared_error', metrics=['mse'])

# vemos la nueva estructura del modelo
model_FT2.summary()

In [ ]:
#entreno de nuevo

# como entrenariamos normalmente
H_model_FT2 = model_FT2.fit(X_train_resized, Y_train,
          batch_size=batch_size,
          validation_split=0.2,
          epochs=20) #aumento las epocas a lo bestia a ver si mejora 

# Evaluamos el modelo
print("[INFO]: Evaluando red neuronal...")
pred_FT2 = model_FT2.predict(X_test_resized, batch_size=128)  #(X) 

model_FT2.summary()

#grafico
sns.set(style="darkgrid")
plt.figure(figsize=(6, 4))
plt.plot(H_model_FT2.history['loss'], label='Train')
plt.plot(H_model_FT2.history['val_loss'], label='Validation')
plt.title('Model_FT2 Loss', fontsize=18)
plt.ylabel('Loss', fontsize=12)
plt.xlabel('Epoch', fontsize=12)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.legend(loc='middle right', fontsize=10)
sns.despine()
plt.show()

In [ ]:
#calculo el mse y el mae del modelo
max_Y = Y_train.max()
pred_FT2_denorm = pred_FT2
y_test_denorm = y_test * max_Y

rmse = mean_squared_error(y_true  = y_test_denorm, y_pred  = pred_FT2_denorm,squared = False)
print(f"El error cuadrático medio (rmse) de test es: {rmse}")

mae = mean_absolute_error(y_test_denorm, pred_FT2_denorm)
print(f"El error aboluto(mae) de test es: {mae}")


##### Aquí me empeora el modelo, no voy a seguir descongelando capas, me quedo con el modelo anterior model_FT con solo el último bloque convolucional descongelado. Muestro los mejores resultados que he obtenido hasta ahora.

In [ ]:
#calculo el mse y el mae del modelo
max_Y = Y_train.max()
pred_FT_denorm = pred_FT
y_test_denorm = y_test * max_Y

rmse = mean_squared_error(y_true  = y_test_denorm, y_pred  = pred_FT_denorm,squared = False)
print(f"El error cuadrático medio (rmse) de test es: {rmse}")

mae = mean_absolute_error(y_test_denorm, pred_FT_denorm)
print(f"El error aboluto(mae) de test es: {mae}")


In [ ]:
# comprobamos el loss en el test set
scores = model_FT.evaluate(X_test_resized, Y_test, verbose=1)
print('Test loss:', scores[0])


##### Voy a probar con hyperopt a buscar los parámetros óptimos

##### Mejoro el modelo usando hyperot

In [ ]:
# instalamos los paquetes necesarios
!pip install networkx==1.11 # para instala hyperopt correctamente, si no, da errores
!pip install hyperopt

In [ ]:
# imports necesarios
import sys
import time
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping

In [ ]:
X_train_resized = np.array(X_train_resized)
X_test_resized = np.array(X_test_resized)

In [ ]:
np.random.seed(42)
# definimos nuestro espacio de búsqueda
# vamos a variar:
# - el número de filtros en nuestras capas conv
# - el porcentaje de dropout
# - el número de neuronas en la capa dense
space = {
    'learning_rate': hp.choice('lr', [0.05, 0.001, 0.0001]),
    'dropout': hp.uniform('dropout', 0.0, 0.5),
    'neurons_dense': hp.choice('neurons_dense', [256, 512, 1024]), 
}

def	get_callbacks(pars):
  callbacks	= [EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=2, verbose=0, mode='auto')]
  return callbacks

def mi_cnn(pars):
  print ('Parameters: ', pars)
  model = Sequential()
  # creamos el modelo base
  base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)

  # vamos a entrenar la ultima capa conv para ver si mejoramos los datos
  for layer in base_model.layers:
    if layer.name == 'block5_conv1':
      break
  layer.trainable = False
  print('Capa ' + layer.name + ' congelada.')


# cogemos la última capa del model y le añadimos nuestro clasificador (top_model)
  last = base_model.layers[-1].output
  x = Flatten()(last)
  x = Dense(pars["neurons_dense"], activation='relu', name='fc1')(x)
  x = Dropout(pars['dropout'])(x)
  x = Dense(1, activation='sigmoid', name='predictions')(x)
  model = Model(base_model.input, x)

# compilamos el modelo
  model.compile(optimizer=Adam(lr=pars["learning_rate"]), loss='mse', metrics=['mse'])

# vemos la nueva estructura del modelo
  model.summary()

  
  # Entrenamos el modelo
  history = model.fit(X_train_resized, 
                      y_train,
                      batch_size=50,
                      shuffle=True,
                      epochs=8,
                      validation_split=0.2,
                      verbose = 0,
                      callbacks = get_callbacks(pars))

  best_epoch_loss = np.argmin(history.history['val_loss'])
  best_val_loss = np.min(history.history['val_loss'])

  
  print('Epoch {}  - val loss: {}'.format(best_epoch_loss, best_val_loss))
  sys.stdout.flush()
  
  return {'loss': best_val_loss, 'best_epoch': best_epoch_loss, 'eval_time': time.time(), 'status': STATUS_OK, 'model': model, 'history': history}


trials = Trials()
best = fmin(mi_cnn, space, algo=tpe.suggest, max_evals=10, trials=trials)
print(best)

In [ ]:
trials.losses()

In [ ]:
print('El mejor conjunto de hiperarámetros tiene un loss: 0.00816590990871191,, y es:') 
print('\n dropout: 0.2031793781184285, learning_rate: 0.0001, neurons_dense: 256\n' )


#### entreno el modelo bueno con los hiperparámetros que me ha dado el hyperot

In [ ]:
#inicializo con una semilla
np.random.seed(42)
input_shape = (224, 224, 3)

bm_cnn_hp = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
bm_cnn_hp.summary()

In [ ]:
# Hago Fine Tuning, Descongelo la ultima capa convolucional
from keras.regularizers import l2

for layer in bm_cnn_hp.layers:
  if layer.name == 'block5_conv1': #descongelo el bloque 5
    break
  layer.trainable = False
  print('Capa ' + layer.name + ' congelada.')

# cogemos la última capa del model y le añadimos nuestro clasificador (top_model)
last = bm_cnn_hp.layers[-1].output
x = Flatten()(last)
x = Dense(256, activation='relu', name='fc1')(x) 
x = Dropout(0.20)(x)
x = Dense(1, activation='linear', name='predictions')(x)#linear multiplica por x1.0, es la indentity function
model_cnn_hp = Model(bm_cnn_hp.input, x)

# compilamos el modelo
model_cnn_hp.compile(optimizer=Adam(lr=0.0001), loss='mse', metrics=['mse'])

# vemos la nueva estructura del modelo
model_cnn_hp.summary()

In [ ]:
#entreno de nuevo

# como entrenariamos normalmente
H_cnn_hp = model_cnn_hp.fit(X_train_resized, Y_train,
          batch_size=batch_size,
          validation_split=0.2,
          epochs=20)

# Evaluamos el modelo
print("[INFO]: Evaluando red neuronal...")
pred_hp = model_cnn_hp.predict(X_test_resized, batch_size=128)  #(X)

model_cnn_hp.summary()

#grafico
sns.set(style="darkgrid")
plt.figure(figsize=(6, 4))
plt.plot(H_cnn_hp.history['loss'], label='Train')
plt.plot(H_cnn_hp.history['val_loss'], label='Validation')
plt.title('Model_FT Loss', fontsize=18)
plt.ylabel('Loss', fontsize=12)
plt.xlabel('Epoch', fontsize=12)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.legend(loc='middle right', fontsize=10)
sns.despine()
plt.show()

In [ ]:
# comprobamos el loss en el test set
scores = model_cnn_hp.evaluate(X_test_resized, Y_test, verbose=1)
print('Test loss:', scores[0])


In [ ]:
#calculo el mse y el mae del modelo
max_Y = Y_train.max()
pred_lm_denorm = pred_hp
y_test_denorm = y_test * max_Y

rmse = mean_squared_error(y_true  = y_test_denorm, y_pred  = pred_lm_denorm,squared = False)
print(f"El error cuadrático medio (rmse) de test es: {rmse}")

mae = mean_absolute_error(y_test_denorm, pred_lm_denorm)
print(f"El error aboluto(mae) de test es: {mae}")


Hyperopt me mejora el mse, a costa de empeorar el mae. Siguen siendo unos valores altos de error.

## Modelo híbrido

In [ ]:
#remember de mis datos
#x_train, train normalizado
#x_test, test normalizado

#y_train, etiquetas norm
#y_test, etiquetas norm

#X_train_resized, imagenes norm
#X_test_resized, imagenes norm

In [ ]:
!pip install -q keras

In [ ]:
!pip install imutils

In [ ]:
# importar los paquetes necesarios
import tensorflow as tf
from tensorflow import keras
from keras.layers import BatchNormalization
from keras.layers import concatenate
from keras.layers import Input, concatenate
from keras.models import Model
from keras.optimizers import Adam
import cv2 
import imutils
#from pyimagesearch import datasets
#from pyimagesearch import models
import argparse
import locale
import os

##### Modelo híbrido básico 

In [ ]:
np.random.seed(42)
#Inputs MLP
inputs_mlp = Input(shape=(9,))
x_mlp1 = Dense(64, activation='relu', bias_initializer='zeros')(inputs_mlp)
x_mlp2 = Dense(3, activation='relu')(x_mlp1)


#Imputs CNN
#inputs_cnn = Input(shape=(224, 224, 3))
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

for layer in base_model.layers:
  if layer.name == 'block5_conv1':
    break
  layer.trainable = False
# cogemos la última capa del model y le añadimos nuestro clasificador (top_model)
last = base_model.layers[-1].output
x1 = Conv2D(9, (3,3), activation='relu')(last)
x2 = MaxPooling2D(pool_size=(2,2)) (x1)  #aquí dejo mis 9 neuronas a 3
x3 = Flatten()(x2) #paso a vector

In [ ]:
#concat of inputs
combinedInput = concatenate([x_mlp2,x3])
# output 
output = Dense (1, activation='linear') (combinedInput)

In [ ]:
#Definir el modelo
model_hib= Model(inputs=[inputs_mlp, base_model.input], outputs=output)


In [ ]:
#optimizador
opt = Adam(lr=1e-3, decay=1e-3 / 20)

# Compilación del modelo
model_hib.compile(loss='mse', optimizer=opt)

In [ ]:
# train the model
H_hib= model_hib.fit([X_train_norm, X_train_resized], y_train, epochs=20, batch_size=128, validation_split=0.2)

In [ ]:
# make predictions on the testing data
print("[INFO] predicting house prices...")
preds_hib= model_hib.predict([x_test, X_test_resized])

diff = preds_hib.flatten() - y_test
percentDiff = (diff / y_test) * 100
absPercentDiff = np. abs(diff)
print(absPercentDiff.mean())

model_hib.summary()

#grafico
sns.set(style="darkgrid")
plt.figure(figsize=(6, 4))
plt.plot(H_hib.history['loss'], label='Train')
plt.plot(H_hib.history['val_loss'], label='Validation')
plt.title('Model_hib Loss', fontsize=18)
plt.ylabel('Loss', fontsize=12)
plt.xlabel('Epoch', fontsize=12)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.legend(loc='middle right', fontsize=10)
sns.despine()
plt.show()

In [ ]:
print('mean: ', np.mean(absPercentDiff))
print('desviación standard: ', np.std(absPercentDiff))


In [ ]:
#desnormalizo el resultado para ver el precio en eur en lo que nos estamos confundiendo
pred_desnorm=preds_hib*max_Y

diff = list(flatten(pred_desnorm)) - (y_test*max_Y)
absPercentDiff = np. abs(diff)
print("El error medio en EUR",absPercentDiff.mean())


In [ ]:
#calculo el mse y el mae del modelo
max_Y = Y_train.max()
pred_hib_denorm = preds_hib
y_test_denorm = y_test * max_Y

rmse = mean_squared_error(y_true  = y_test_denorm, y_pred  = pred_hib_denorm,squared = False)
print(f"El error cuadrático medio (rmse) de test es: {rmse}")

mae = mean_absolute_error(y_test_denorm, pred_hib_denorm)
print(f"El error aboluto(mae) de test es: {mae}")


#### Los valores son muy malos, voy a ver que resultados me da usando flatten como en mi CNN

In [ ]:
np.random.seed(42)
#Inputs MLP
inputs_mlp1 = Input(shape=(9,)) 
x1_mlp1 = Dense(64, activation='relu',kernel_initializer='random_uniform', bias_initializer='zeros')(inputs_mlp)
x1_mlp2 = Dense(3, activation='relu')(x1_mlp1)

#Imputs CNN
base_model1 = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

for layer in base_model1.layers:
  if layer.name == 'block5_conv1':
    break
  layer.trainable = False
# cogemos la última capa del model y le añadimos nuestro clasificador (top_model)
last = base_model1.layers[-1].output
x1 = Flatten()(last)
x2 = Dense(1000, activation='relu')(x1)
x3 = Dense(128, activation='relu', name='fc1')(x2)
x4 = Dropout(0.3)(x3)
x5 = Dense(3, activation='relu')(x4)

In [ ]:
#concat of inputs
combinedInput1 = concatenate([x1_mlp2,x5])
# output 
output1 = Dense (1, activation='linear') (combinedInput1)

#Definir el modelo
model_hib1= Model(inputs=[inputs_mlp, base_model1.input], outputs=output1)


In [ ]:
#optimizador
opt1 = Adam(lr=1e-3, decay=1e-3 / 20)

# Compilación del modelo
model_hib1.compile(loss='mse', optimizer=opt1)

In [ ]:
# train the model
H_hib1= model_hib1.fit([X_train_norm, X_train_resized], y_train, epochs=20, batch_size=128, validation_split=0.2)

In [ ]:
# make predictions on the testing data
print("[INFO] predicting house prices...")
preds_hib1= model_hib1.predict([x_test, X_test_resized])

diff1 = preds_hib1.flatten() - y_test
percentDiff1 = (diff1 / y_test) * 100
absPercentDiff1 = np. abs(diff1)
print(absPercentDiff1.mean())

model_hib1.summary()

#grafico
sns.set(style="darkgrid")
plt.figure(figsize=(6, 4))
plt.plot(H_hib1.history['loss'], label='Train')
plt.plot(H_hib1.history['val_loss'], label='Validation')
plt.title('Model_Hib1 Loss', fontsize=18)
plt.ylabel('Loss', fontsize=12)
plt.xlabel('Epoch', fontsize=12)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.legend(loc='middle right', fontsize=10)
sns.despine()
plt.show()

In [ ]:
print('mean: ', np.mean(absPercentDiff1))
print('desviación standard: ', np.std(absPercentDiff1))


In [ ]:
#desnormalizo el resultado para ver el precio en eur en lo que nos estamos confundiendo
pred_desnorm1=preds_hib1*max_Y

diff1 = list(flatten(pred_desnorm1)) - (y_test*max_Y)
absPercentDiff1 = np. abs(diff1)
print("El error medio en EUR",absPercentDiff1.mean())


#### He obtenido mejores resultados utilizando densas y añadiento un dropout, ya que el maxpooling lo lleva el VGG16, creo que es más óptimo usar Dropout entre las densas. Considero que un error de 28 no es un valor muy alto por lo que aquí finalizo el estudio